In [ ]:
import tensorflow as tf
from ts_ver1 import (data_proc, univariate_data, 
                     show_plot, baseline, 
                     lstm_model, multivariate_data,
                     single_step_model,
                     multi_step_model)

import matplotlib.pyplot as plt
tf.random.set_seed(13)

In [ ]:
df = data_proc()

In [ ]:
df.head()

In [ ]:
TRAIN_SPLIT = 300000


# Part 1: Forecast a univariate time series

In [ ]:
uni_data = df['T (degC)']
uni_data.index = df['Date Time']
uni_data.head()

In [ ]:
uni_data.plot(subplots=True);

In [ ]:
uni_data = uni_data.values

uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()

uni_data = (uni_data-uni_train_mean)/uni_train_std


univariate_past_history = 20
univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)

x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

In [ ]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target temperature to predict')
print (y_train_uni[0])

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Sample Example');

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0], baseline(x_train_uni[0])], 0,
           'Baseline Prediction Example');

In [ ]:
simple_lstm_model = lstm_model(x_train_uni, y_train_uni, x_val_uni, y_val_uni)

# Multivariate forecast with lstm

In [ ]:
features_considered = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']

features = df[features_considered]
features.index = df['Date Time']
features.head()

In [ ]:
features.plot(subplots=True);

In [ ]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)

dataset = (dataset - data_mean)/data_std

# Single step model

In [ ]:
# data for the last 5 days is used (720 observations) predict 12h ahead
past_history = 720
future_target = 72
STEP = 6

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)

x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [ ]:
print ('Single window of past history : {}'.format(x_train_single[0].shape))

In [ ]:
trained_model, single_step_history = single_step_model(x_train_single, y_train_single, x_val_single, y_val_single)

In [ ]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [ ]:
plot_train_history(single_step_history,
                   'Single Step Training and validation loss')

# Multi-Step model

In [ ]:
# we want to predict 12 hours ahead, 
# that means 72 hours
future_target = 72
x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 1], 0, TRAIN_SPLIT, 
                                                 past_history, future_target, STEP)
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 1], TRAIN_SPLIT, 
                                             None, past_history, future_target, STEP)

print(f'Single window of past history: {x_train_multi[0].shape}')
print(f'Target temperature to predict: {y_train_multi[0].shape}')

In [ ]:
trained_model, multi_step_history = multi_step_model(x_train_multi, y_train_multi, 
                     x_val_multi, y_val_multi, epochs=10)

# Hyperparam search

In [ ]:
from ts_ver1 import train_val_tf, TimeSeriesLSTM

In [ ]:
# train_data_single, val_data_single = train_val_tf(
#     x_train_single, y_train_single,
#     x_val_single, y_val_single)

# data for the last 5 days is used (720 observations) predict 12h ahead
past_history = 720
future_target = 72
STEP = 6

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)

x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

In [ ]:
from kerastuner.tuners import RandomSearch, Hyperband

hypermodel = TimeSeriesLSTM(num_outputs=1)
    
tuner = Hyperband(
    hypermodel,
    objective='mae',
    #max_trials=40,
    directory='my_dir',
    project_name='helloworld', 
    max_epochs=5
)
tuner.search(x_train_single, y_train_single, epochs=20, validation_data=(x_val_single, y_val_single))